# Анализ текстов с SVM

###### Импорт необходимых библиотек

In [4]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold

###### Импорт датасета, разделение на признаки $X$ и целевую переменную $y$

In [5]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [7]:
X = newsgroups.data # - признаки, представляющие собой строки из новостных лент
y = newsgroups.target

In [15]:
# пример признака до разбиения на отдельные слова
X[0]

'From: 9051467f@levels.unisa.edu.au (The Desert Brat)\nSubject: Re: Keith Schneider - Stealth Poster?\nOrganization: Cured, discharged\nLines: 24\n\nIn article <1pa0f4INNpit@gap.caltech.edu>, keith@cco.caltech.edu (Keith Allan Schneider) writes:\n\n> But really, are you threatened by the motto, or by the people that use it?\n\nEvery time somone writes something and says it is merely describing the norm,\nit is infact re-inforcing that norm upon those programmed not to think for\nthemselves. The motto is dangerous in itself, it tells the world that every\n*true* American is god-fearing, and puts down those who do not fear gods. It\ndoesn\'t need anyone to make it dangerous, it does a good job itself by just\nexisting on your currency.\n\n> keith\n\nThe Desert Brat\n-- \nJohn J McVey, Elc&Eltnc Eng, Whyalla, Uni S Australia,    ________\n9051467f@levels.unisa.edu.au      T.S.A.K.C.            \\/Darwin o\\\nFor replies, mail to whjjm@wh.whyalla.unisa.edu.au      /\\________/\nDisclaimer:

###### Обучение линейной модели SVM с параметрами

In [16]:
# Вычисление TF-IDF для слов в строках признаков
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

In [23]:
print(f'В новой таблице признаков находится {X_vectorized.shape[0]} строк (новостных лент)')
print(f'В новой таблице признаков находится {X_vectorized.shape[1]} столбца (различных слов в новостных лентах)')

В новой таблице признаков находится 1786 строк (новостных лент)
В новой таблице признаков находится 28382 столбца (различных слов в новостных лентах)


In [26]:
# создание сетки параметров и стратегии кросс-валидации
param_grid = {
    'C': np.power(10.0, np.arange(-5, 6))
}
cv = KFold(n_splits=5, shuffle=True, random_state=241)

In [28]:
# Поиск наилучшего значения константы C для SVM
estimators = GridSearchCV(SVC(kernel='linear', random_state=241), param_grid, scoring='accuracy', cv=cv)
estimators.fit(X_vectorized, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [43]:
for a in zip(estimators.cv_results_['param_C'],estimators.cv_results_['mean_test_score']):
    print(f'C = {a[0]}', ', ', f'средняя ошибка равна = {round(a[1], 4)}', sep='')

C = 1e-05, средняя ошибка равна = 0.5526
C = 0.0001, средняя ошибка равна = 0.5526
C = 0.001, средняя ошибка равна = 0.5526
C = 0.01, средняя ошибка равна = 0.5526
C = 0.1, средняя ошибка равна = 0.9502
C = 1.0, средняя ошибка равна = 0.9933
C = 10.0, средняя ошибка равна = 0.9933
C = 100.0, средняя ошибка равна = 0.9933
C = 1000.0, средняя ошибка равна = 0.9933
C = 10000.0, средняя ошибка равна = 0.9933
C = 100000.0, средняя ошибка равна = 0.9933


Таким образом, наименьшим оптимальным параметром $С$ для текущей выборки является значение $С = 1.0$

In [44]:
# обучение оптимальной модели
svm = SVC(C=1.0, kernel='linear', random_state=241)
svm.fit(X_vectorized, y)

SVC(kernel='linear', random_state=241)

In [65]:
# Получение весов и их сортировка для последующего отбора 10 наибольших весов по модулю
coefs = abs(svm.coef_.todense().A1)
coefs_indices = np.argsort(coefs)
coefs_10_max = coefs_indices[-10 : ]

In [76]:
# Получение списка со словами,имеющими наибольший абсолютный вес (больше всего встречающиеся в выборке)
feature_mapping = vectorizer.get_feature_names_out()
most_weight_features = []
for coef in coefs_10_max:
    most_weight_features.append(feature_mapping[coef])
most_weight_features = sorted(most_weight_features)
print('\033[1mСписок 10 слов, имеющих наибольший вес по модулю:\033[0m')
for n in range(10):
    if n == 9:
        print(most_weight_features[n])
        break
    print(f'{most_weight_features[n]}', end=', ')

Список 10 слов, имеющих наибольший вес по модулю:
atheism, atheists, bible, god, keith, moon, religion, sci, sky, space
